In [165]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

from glob import glob

In [166]:
import unicodedata
import numpy as np
# Function to normalize the names
def normalize_municipality_name(name):
    if type(name) == float:
        return name
    else:
        # Normalize the string (remove diacritical marks)
        name_without_diacritics = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
        # Convert to lowercase
        return name_without_diacritics.lower()



In [167]:
print(os.getcwd())

/Users/dhruvnovaims/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/climate-econometrics/countries/portugal/purchases_paper/notebooks/data_proc


### Import Municipality Data

In [170]:
# Directory containing the files
data_dir = r'../../../../portugal/purchases_paper/datasets/ine/'

# Load municipality metadata
df_mun_metadata = pd.read_excel(r'../../../../portugal/municipality_data/concelhos-metadata.xlsx')
df_mun_metadata['designacao'] = df_mun_metadata['designacao'].map(normalize_municipality_name)
mun_gdf = gpd.read_file(r'../../../../portugal/municipality_data/municipalities-shapefile-2/concelhos.shp')
mun_gdf.CCA_2 = mun_gdf.CCA_2.astype('int64')
#mun_metadata = df_mun_metadata[['dicofre','designacao']]
mun_gdf = mun_gdf.merge(df_mun_metadata, left_on='CCA_2', right_on='dicofre')
mun_gdf = mun_gdf.rename(columns={'designacao': 'concelho'})
mainland = mun_gdf[~mun_gdf['NAME_1'].isin(['Azores', 'Madeira'])]
# Grouping and dissolving the municipalities by district

municipalities_mainland = mainland[['concelho', 'HASC_2','CCA_2', 'geometry']].copy()
municipalities_mainland = municipalities_mainland.rename(columns={'HASC_2' : 'district_id'})



In [171]:
municipalities_mainland.concelho.nunique()

278

In [172]:
# Normalize municipality names for filtering
valid_municipalities = municipalities_mainland.concelho.unique()


### Clean Purchases Data

In [173]:

df_purchases_1 = pd.read_excel(f"{data_dir}ine_purchases_euros.xls", skiprows=9, skipfooter=12, dtype=str)
df_purchases_2 = pd.read_excel(f"{data_dir}ine_purchases_euros2.xls", skiprows=9, skipfooter=12, dtype=str)

In [174]:
geo_col = 'Geographic localization (NUTS - 2013)'
df_purchases_1 = df_purchases_1.loc[:, ~df_purchases_1.columns.str.startswith('Unnamed:')]
df_purchases_1 = df_purchases_1.drop(columns=[f"{geo_col} Code"])
# Determine the geo column by searching for column names that start with the desired strings.

df_purchases_1.rename(columns={geo_col: 'concelho'}, inplace=True)

# Normalize municipality names
df_purchases_1['concelho'] = df_purchases_1['concelho'].map(normalize_municipality_name)

# Filter for valid municipalities
df_purchases_1 = df_purchases_1[df_purchases_1['concelho'].isin(valid_municipalities)]
# Melt the DataFrame into long format with custom column name
df_purchases_1 = df_purchases_1.melt(id_vars=['concelho'], var_name='time', value_name='purchases')
df_purchases_1['time'] = pd.to_datetime(df_purchases_1['time'], format='%B %Y').dt.strftime('%Y-%m')


In [175]:
geo_col = 'Geographic localization (NUTS - 2013)'
df_purchases_2 = df_purchases_2.loc[:, ~df_purchases_2.columns.str.startswith('Unnamed:')]
df_purchases_2 = df_purchases_2.drop(columns=[f"{geo_col} Code"])
# Determine the geo column by searching for column names that start with the desired strings.

df_purchases_2.rename(columns={geo_col: 'concelho'}, inplace=True)

# Normalize municipality names
df_purchases_2['concelho'] = df_purchases_2['concelho'].map(normalize_municipality_name)

# Filter for valid municipalities
df_purchases_2 = df_purchases_2[df_purchases_2['concelho'].isin(valid_municipalities)]
# Melt the DataFrame into long format with custom column name
df_purchases_2 = df_purchases_2.melt(id_vars=['concelho'], var_name='time', value_name='purchases')
df_purchases_2['time'] = pd.to_datetime(df_purchases_2['time'], format='%B %Y').dt.strftime('%Y-%m')

In [176]:
df_purchases = pd.concat([df_purchases_1, df_purchases_2])
# Convert all numeric values to float and fill NaN with 0
df_purchases['purchases'] = df_purchases['purchases'].astype(float)


In [177]:
df_purchases['purchases_log'] = np.log(df_purchases['purchases'])

### Clean Economic Data

In [179]:
econ_data_df = pd.read_stata(r'/Users/dhruvnovaims/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/msc-thesis/database/Quarterly_dataset_old.dta') #pd.read_excel(r'../../database/excel/quarterly/Quarterly_dataset_NEW.xlsx')
econ_data_df = econ_data_df[['date', 'disp_inc_conc', 'hpi', 'ur', 'ir_newloans', 'DsgDistrito', 'concelho', 'nuts3']].copy()
econ_data_df_mod = econ_data_df.rename(columns={'date': 'time', 'DsgDistrito' : 'NAME_1', 'concelho' : 'NAME_2'})
econ_data_df_mod['NAME_2'] = econ_data_df_mod['NAME_2'].map(normalize_municipality_name)

econ_data_df_mod.time = econ_data_df_mod.time.dt.to_period('Q')

In [181]:
df_purchases.time = pd.to_datetime(df_purchases.time)
df_purchases_q = (
    df_purchases
    .reset_index()
    .assign(time=lambda x: pd.to_datetime(x['time']).dt.to_period('Q').dt.to_timestamp())
    .groupby(['time', 'concelho'])
    .mean().reset_index()
)


In [ ]:
df_purchases_q.time = df_purchases_q.time.dt.to_period('Q')
df_purchases_q = df_purchases_q.drop(columns=['index'])

In [ ]:
df_econ = econ_data_df_mod[econ_data_df_mod.time.isin(df_purchases_q['time'])]
df_econ = df_econ.rename(columns={'NAME_2' : 'concelho'})
df_econ = df_econ.merge(df_purchases_q, on=['concelho', 'time'])
df_econ['hpi_log'] = np.log(df_econ['hpi'])
df_econ = df_econ[['time', 'concelho', 'hpi', 'hpi_log', 'purchases', 'purchases_log', 'ur']]

In [197]:
df_econ

,time,concelho,hpi,hpi_log,purchases,purchases_log,ur
0,2017Q4,agueda,107.294487,4.675577,1.017530e+07,16.126074,0.036923
1,2014Q4,agueda,93.999625,4.543291,8.095330e+06,15.899491,0.069500
2,2011Q3,agueda,100.198538,4.607154,7.088932e+06,15.773207,0.066504
3,2014Q3,agueda,90.651598,4.507024,7.971248e+06,15.889204,0.071476
4,2010Q3,agueda,100.243610,4.607603,7.039760e+06,15.766467,0.066884
...,...,...,...,...,...,...,...
12227,2018Q4,vouzela,109.446017,4.695431,1.001831e+06,13.816209,0.063996
12228,2014Q3,vouzela,97.002670,4.574739,9.275167e+05,13.716543,0.081636
12229,2010Q3,vouzela,112.399774,4.722062,7.701783e+05,13.545109,0.045091
12230,2011Q2,vouzela,100.567631,4.610830,6.231537e+05,13.342147,0.045419


In [199]:
df_econ.to_excel(r'../../../../portugal/purchases_paper/datasets/economic_data_q.xlsx')